In [252]:
import math
import numpy
import torch
import torchaudio
torchaudio.set_audio_backend("soundfile")
import matplotlib.pyplot as plt

In [253]:
loadedData = torchaudio.load("Samples/a2.wav")
tensor = loadedData[0][0]
sampleRate = loadedData[1]
del loadedData

In [254]:
expectedPitch = 95.
searchRange = 0.2
batchSize = math.floor((1. + searchRange) * sampleRate / expectedPitch)
lowerSearchLimit = math.floor((1. - searchRange) * sampleRate / expectedPitch)

In [255]:
batchStart = 0
deltas = torch.tensor([], dtype = int)
while batchStart + batchSize <= tensor.size()[0] - batchSize:
    
    sample = torch.index_select(tensor, 0, torch.linspace(batchStart, batchStart + batchSize, batchSize, dtype = int))
    zeroTransitions = torch.tensor([], dtype = int)
    for i in range(lowerSearchLimit, batchSize):
        if (sample[i-1] < 0) and (sample[i] > 0):
            zeroTransitions = torch.cat([zeroTransitions, torch.tensor([i])], 0)
            
    error = math.inf
    delta = math.floor(sampleRate / expectedPitch)
    for i in zeroTransitions:
        shiftedSample = torch.index_select(tensor, 0, torch.linspace(batchStart + i.item(), batchStart + batchSize + i.item(), batchSize, dtype = int))
        newError = torch.sum(torch.pow(sample - shiftedSample, 2))
        if error > newError:
            delta = i.item()
            error = newError
            
    deltas = torch.cat([deltas, torch.tensor([delta])])
    batchStart += delta
nBatches = deltas.size()[0]
borders = torch.zeros(nBatches + 1, dtype = int)
for i in range(nBatches):
    borders[i+1] = borders[i] + deltas[i]
avgDelta = torch.mean(deltas.float()).int()

In [256]:
Window = torch.hann_window(avgDelta * 3)
Signals = torch.stft(tensor, 3 * avgDelta, hop_length = avgDelta, win_length = 3 * avgDelta, window = Window, return_complex = True)
Signals = torch.transpose(Signals, 0, 1)
"""
Spectrums = Signals.abs()
#implement better envelope calculation method
Spectrums = torch.roll(Spectrums, -2, dims = 1) + torch.roll(Spectrums, -1, dims = 1) + Spectrums + torch.roll(Spectrums, 1, dims = 1) + torch.roll(Spectrums, 2, dims = 1)
Spectrums = Spectrums / 5.
Spectrum = torch.mean(Spectrums, 0)
"""
Waveform = Signals.abs()
Spectrums = torch.full_like(Waveform, -float("inf"), dtype=torch.float)
for i in range(1000):
    Waveform = torch.max(Waveform, Spectrums)
    
    #Spectrums = torch.fft.ifft(Waveform).abs()
    #Spectrums = torchaudio.functional.lowpass_biquad(Spectrums, sampleRate, sampleRate/avgDelta)
    #Spectrums = torch.fft.fft(Spectrums).real
    
    Spectrums = torch.roll(Waveform, -2, dims = 1) + torch.roll(Waveform, -1, dims = 1) + Waveform + torch.roll(Waveform, 1, dims = 1) + torch.roll(Waveform, 2, dims = 1)
    Spectrums = Spectrums / 5.

Spectrum = torch.mean(Spectrums, 0)
Excitations = torch.empty_like(Signals)
for i in range(Spectrums.size()[0]):
    Excitations[i] = Signals[i] / Spectrums[i]
    Spectrums[i] = Spectrums[i] - Spectrum
Excitations = torch.transpose(Excitations, 0, 1)
Excitation = torch.istft(Excitations, 3 * avgDelta, hop_length = avgDelta, win_length = 3 * avgDelta, window = Window, onesided=True, )

In [285]:
ReturnSignal = torch.stft(Excitation, 3 * avgDelta, hop_length = avgDelta, win_length = 3 * avgDelta, window = Window, return_complex = True)
ReturnSignal = torch.transpose(ReturnSignal, 0, 1)
for i in range(Spectrums.size()[0]):
    ReturnSignal[i] = ReturnSignal[i] * (Spectrum + 5 * torch.roll(Spectrums, 0, dims=0)[i])
ReturnSignal = torch.transpose(ReturnSignal, 0, 1)
ReturnSignal = torch.istft(ReturnSignal, 3 * avgDelta, hop_length = avgDelta, win_length = 3 * avgDelta, window = Window, onesided=True, )

In [286]:
torchaudio.save("output_Averaged.wav", torch.unsqueeze(ReturnSignal, 0), sampleRate, format="wav", encoding="PCM_S", bits_per_sample=32)